## **Transfer learning**

- when we are using pretrained model we must watch to domain and task of our own problem and pretrained model<br>
Full documentation link is here: https://www.learndatasci.com/tutorials/hands-on-transfer-learning-keras/

- github repo link: https://github.com/gabrielcassimiro17/object-detection/blob/main/transfer_learning.ipynb

- **Example :**VGG16 train on imagenet dataset contain 14million hight resolution images and 1000 class/label
Our goal is classification(flower) but pretained model VGG16 cannot contain flower images
- The VGG16 network was not trained to classify different kinds of flowers.

## **Install required libraries**

In [ ]:
!pip install tensorflow_datasets tensorflow keras

## **Load dataset from Tensorflow**


In [4]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [6]:
train_labels

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

1.   We use Include_top=False to remove the classification layer that was trained on the ImageNet dataset and set the model as not trainable
2.   Also, we used the preprocess_input function from VGG16 to normalize the input data.



In [7]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

##load VGG16 model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False   # not trainable weights

# preprocess the input
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

58889256/58889256 [==============================] - 0s 0us/step


**check model summary**


In [8]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

####**Two main points:**
- the model has over 14 Million trained parameters
- and ends with a maxpooling layer that belongs to the Feature Learning part of the network.

Now we add the last layers for our specific problem.<br>
"cutt-off" --> the fullyconnect layers which are called top model

In [9]:
from tensorflow.keras import layers, models

flatten_layers = layers.Flatten()
dense_layer1 = layers.Dense(50, activation = 'relu')
dense_layer2 = layers.Dense(20, activation = 'relu')
prediction_layers = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layers,
    dense_layer1,
    dense_layer2,
    prediction_layers
])

**Add compile and fit the model**

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)


# add EarlyStopping layers--> this layers monitor val_accuracy  if it maximum then stop our model training
# and store weights

early_stopping = EarlyStopping(monitor = 'val_accuracy', mode ="max", patience=5, restore_best_weights=True)



**Strat model training by using callbacks for monitoring**

In [16]:
model.fit(train_ds, train_labels, epochs=10, validation_split=0.2,  callbacks=[early_stopping], batch_size=32)

Epoch 1/10
65/65 [==============================] - 18s 127ms/step - loss: 2.1347 - accuracy: 0.5197 - val_loss: 1.0951 - val_accuracy: 0.5992
Epoch 2/10
65/65 [==============================] - 6s 87ms/step - loss: 0.6983 - accuracy: 0.7411 - val_loss: 1.0174 - val_accuracy: 0.6323
Epoch 3/10
65/65 [==============================] - 6s 88ms/step - loss: 0.4311 - accuracy: 0.8380 - val_loss: 0.9681 - val_accuracy: 0.6576
Epoch 4/10
65/65 [==============================] - 5s 83ms/step - loss: 0.2996 - accuracy: 0.8891 - val_loss: 1.0389 - val_accuracy: 0.6907
Epoch 5/10
65/65 [==============================] - 6s 88ms/step - loss: 0.2190 - accuracy: 0.9299 - val_loss: 1.0165 - val_accuracy: 0.6965
Epoch 6/10
65/65 [==============================] - 6s 87ms/step - loss: 0.1647 - accuracy: 0.9426 - val_loss: 1.0601 - val_accuracy: 0.6907
Epoch 7/10
65/65 [==============================] - 6s 87ms/step - loss: 0.1273 - accuracy: 0.9572 - val_loss: 1.1492 - val_accuracy: 0.6965
Epoch 8/10


## **Evaluating this model on test data**

In [17]:
model.evaluate(test_ds, test_labels)

35/35 [==============================] - 3s 95ms/step - loss: 0.0541 - accuracy: 0.9855


[0.054106153547763824, 0.9854677319526672]

## **Train the CNN model from scratch on the same dataset and compare it with pre-train model**

In [20]:
from tensorflow.keras import Sequential , layers
from tensorflow.keras.callbacks import EarlyStopping  # for monitoring
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [23]:
own_model = Sequential()

own_model.add(Rescaling(1./255, input_shape=(150,150,3)))

own_model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
own_model.add(layers.MaxPooling2D(3))


own_model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
own_model.add(layers.MaxPooling2D(2))

own_model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
own_model.add(layers.MaxPooling2D(2))

own_model.add(layers.Flatten())
own_model.add(layers.Dense(50, activation='relu'))
own_model.add(layers.Dense(20, activation='relu'))
own_model.add(layers.Dense(5, activation = 'softmax'))


In [26]:

own_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

own_model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])


Epoch 1/50
65/65 [==============================] - 6s 30ms/step - loss: 1.5768 - accuracy: 0.2545 - val_loss: 1.4958 - val_accuracy: 0.3444
Epoch 2/50
65/65 [==============================] - 2s 25ms/step - loss: 1.3540 - accuracy: 0.3981 - val_loss: 1.2658 - val_accuracy: 0.4864
Epoch 3/50
65/65 [==============================] - 1s 21ms/step - loss: 1.2428 - accuracy: 0.4545 - val_loss: 1.2525 - val_accuracy: 0.4981
Epoch 4/50
65/65 [==============================] - 1s 23ms/step - loss: 1.1755 - accuracy: 0.5007 - val_loss: 1.1797 - val_accuracy: 0.4942
Epoch 5/50
65/65 [==============================] - 1s 23ms/step - loss: 1.1346 - accuracy: 0.5139 - val_loss: 1.1302 - val_accuracy: 0.5195
Epoch 6/50
65/65 [==============================] - 1s 23ms/step - loss: 1.0356 - accuracy: 0.5586 - val_loss: 1.1010 - val_accuracy: 0.5331
Epoch 7/50
65/65 [==============================] - 1s 20ms/step - loss: 0.9630 - accuracy: 0.6078 - val_loss: 1.1968 - val_accuracy: 0.5311
Epoch 8/50
65

## **Again evaluate our own train model that we are train from scratch**

In [27]:
own_model.evaluate(test_ds, test_labels)

35/35 [==============================] - 1s 15ms/step - loss: 0.4236 - accuracy: 0.8520


[0.42360004782676697, 0.8519527912139893]

- Compare the result of own train model that we made it from scrach and result of using pre-traine model<br>
-The result of pre-trian model is better the model which are made from scratch